# Day 17
## Part 1

In [5]:
import parse

def parse_data(s):
    return parse.parse('target area: x={min_x:d}..{max_x:d}, y={min_y:d}..{max_y:d}', s).named

test_data = parse_data('target area: x=20..30, y=-10..-5')
test_data

{'min_x': 20, 'max_x': 30, 'min_y': -10, 'max_y': -5}

Assume that as $x$ will get to $0$ and stay there, $y$ will continue rising and fall while $x$ is constant.

$x$ and $y$ are independent, so find the initial velocities of $x$ that reach 0 between `min_x` and `max_x` and how many steps it takes them to do so. An initial velocity of $v_x$ will reach 0 after $v_x$ steps and will have an $x$ coordinate of $v_x(v_x+1)/2$, so the bounds for the velocities are given by $n^2 + n - 2x_{min}=0$ and $n^2 + n - 2x_{max}=0$. Solving the polynomials gives the following.

In [21]:
import math

def possible_x_velocities(min_x, max_x):
    return math.ceil((-1 + math.sqrt(1 + 8 * min_x)) / 2), math.floor((-1 + math.sqrt(1 + 8 * max_x)) / 2)
    

In [30]:
possible_x_velocities(20, 30)

(6, 7)

I've just realised we don't really care what $v_x$ is. Never mind.

A starting $y$ velocity of $v_y$ means $y$ reach its apex $v_y(v_y+1)/2$ after $v_y$ steps, and then takes another $v_y$ steps to return to 0, where it has a velocity of $-y_v$. 

Oh hang on, we then need to work out the highest value of $v_y$ such that its next step $-v_y-1$ is within the target area, which is 9 for the example above, and $9(9+1)/2 = 45$.

My data is `target area: x=111..161, y=-154..-101`, so

In [27]:
153*154//2

11781

is the answer.

## Part 2

Now need to find the number of pairs of velocities that land in the target at any point. For each number of steps count the number of initial $v_x$s and the number of initial $v_y$s that are in the target at that point, and multiply them together as they're independent. 

Possible values of $v_x$ are bound by the minimum value returned by the `possible_x_velocities` function - it's actually useful! - and $x_{max}$. $v_y$ is bound between $y_{min}$ and $-y_{min}-1$, with a gap in the middle that I can't be bothered working out.

The $x$ values bounded by the `possible_x_velocities` function will always be there, so adjustments need to be made for that.

In [41]:
from collections import defaultdict
from itertools import product

def part_2(data):
    min_x, max_x, min_y, max_y = data['min_x'], data['max_x'], data['min_y'], data['max_y']
    n_steps_xs = defaultdict(set)
    static_xs = possible_x_velocities(min_x, max_x)
    for vx in range(static_xs[0], max_x + 1):
        x = 0
        v = vx
        n = 0
        while x < max_x and v > 0:
            x += v
            v -= 1
            n += 1
            if min_x <= x <= max_x:
                n_steps_xs[n].add(vx)
    
    n_steps_ys = defaultdict(set)
    for vy in range(min_y, -min_y):
        y = 0
        v = vy
        n = 0
        while y > min_y:
            y += v
            v -= 1
            n += 1
            if min_y <= y <= max_y:
                n_steps_ys[n].add(vy)
                
    for vx in static_xs:
        for n in n_steps_ys:
            if n > vx:
                n_steps_xs[n].add(vx)

    velocities = set()
    for n in n_steps_ys:
        for vx, vy in product(n_steps_xs[n], n_steps_ys[n]):
            velocities.add((vx, vy))

    return len(velocities)

assert part_2(test_data) == 112

In [42]:
data = parse_data('target area: x=111..161, y=-154..-101')
part_2(data)

4403

Another one where the test data passes but the real data doesn't.